# Object References, Mutability, and Recyling

## Identify, Equality, and Alias

In [1]:
chales = {'name': 'Chales L. Dodgson', 'born': 1832,  'blance': 950}

In [2]:
lewis = chales

In [3]:
id(chales), id(lewis)

(4447597824, 4447597824)

In [4]:
chales is lewis

True

In [5]:
alex = {'name': 'Chales L. Dodgson', 'born': 1832,  'blance': 950}

In [6]:
chales == alex

True

In [7]:
chales is alex

False

lewis和chales是别名，即两个变量绑定同一个对象。而alex不是chales的别名，因为二者绑定的不是同的对象。alex和chales绑定对象具有相同的值(==比较的就是值)，但是它们的标识不同。

*https://docs.python.org/3/reference/datamodel.html#objects-values-and-types*

每个变量都有标识、类型和值。对象一旦创建，它的标识绝不会变；你可以把标识理解为对象在内存中地址。is运算符比较两个对象的标识；id()函数返回对象标识的整数表示。

编程中很少使用id()函数，标识最常用is运算符检查。

### Choosing Between == and is

**==运算符比较两个对象的值（对象中保存的数据），而is比较两个对象的标识。**

在变量和单例值之间比较时，因该使用is。

In [8]:
x = None
x is None

True

is 运算符比 ==运算符快，因为它不能重载，所以Python不用寻找并调用特殊方法，而是直接比较两个整数ID。而a==b是语法糖，等同于`a.__eq__(b)`。继承自object的`__eq__`方法比较两个对象的ID，结果与is一样。但是多数内置类型使用更有意义的方式覆盖了`__eq__`方法，会考虑对象的属性的值。

### The Relative Immutibility of Tuples

元组与多数Python集合一样，保存的是对象引用。如果引用的元素是可变的，即便元组本身不变，元素依然可变。也就是说，元组的不可变性其实是指tuple数据结构的物理不可变，与引用的对象无关。

## Copies Are Shallow by Default

构造方法或`[:]`做的浅复制。 

In [9]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)
l2

[3, [55, 44], (7, 8, 9)]

In [10]:
l1 == l2

True

In [11]:
l2 is l1

False

http://www.pythontutor.com/visualize.html#code=l1%20%3D%20%5B3,%20%5B55,%2044%5D,%20%287,%208,%209%29%5D%0Al2%20%3D%20list%28l1%29&cumulative=false&curInstr=1&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false

## Function Parameters as References

Python唯一支持的参数传递模式共享传参(call by sharing)。共享传参是指函数的各个形式参数获得实参中各个引用的副本。也就是说，函数内部的行参是实参的别名。

```
The only mode of parameter passing in Python is call by sharing. Call by sharing means that each formal parameter of the function get a copy of each of each reference in the arguments. In other words, the parameters inside the function become alias of the actual arguments.
```

In [12]:
def f(a, b):
    a += b
    return a

In [13]:
x = 1
y = 2
f(x, y), x, y

(3, 1, 2)

In [14]:
a = [1, 2]
b = [3, 4]
f(a, b), a, b

([1, 2, 3, 4], [1, 2, 3, 4], [3, 4])

In [15]:
t = (10, 20)
u = (30, 40)
f(t, u), t, u

((10, 20, 30, 40), (10, 20), (30, 40))

### Mutable Types as Parameter Defaults: Bad Idea

> **Note: Default parameters can not be mutable types.**

In [16]:
class HauntedBus:
    
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

In [17]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [18]:
bus1.pick('Charlie')
bus1.drop('Alice')

In [19]:
bus1.passengers

['Bill', 'Charlie']

In [20]:
bus2 = HauntedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [21]:
bus3 = HauntedBus()
bus3.passengers

['Carrie']

In [22]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [23]:
bus3.passengers is bus2.passengers

True

In [25]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [26]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

**可变默认值导致的这个问题说明了为什么通常使用None作为接收可变值的参数的默认值。**

### Defensive Programming with Mutable Parameters

In [27]:
class TwillightBus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers
            
    def pick(self, name):
        self.passengers.append(name)
        
    def drop(self, name):
        self.passengers.remove(name)

## del and Garbage Collection

```
Objects are never explicitly destroyed; however, when they become unreachable they may be garbage-collected.
```

del语句删除名称，而不是对象。当删除的变量保存的是对象的最后一个引用，或者无法得到对象时，del命令可能会导致对象被当作垃圾回收。重新绑定也可能会导致对象的引用计数归零，导致对象被销毁。

```
The del statement delete names, not objects. An object may be garbage collected as result of a del command, but only if the varaible deleted holds the last reference to the object, or if the object becomes unreachable.
```


在CPython中，垃圾回收使用主要算法是引用计数。实际上，每个对象都会统计有多引用指向自己。引用计数归零时，对象立即被销毁：CPython会在对象上调用`__del__`方法，然后释放分配给对象的内存。CPython2.0增加了分代回收算法，用于检测循环引用中涉及的对象组--如果一组对象之间全是相互饮用，即使再出色的引用方式也会导致数组中对象不可获取。

In [44]:
import weakref

In [45]:
s1 = {1, 2, 3}
s2 = s1

In [46]:
def bye():
    print('Gone with the wind...')

In [47]:
ender = weakref.finalize(s1, bye)

In [48]:
ender.alive

True

In [49]:
del s1

In [50]:
s2 = 'spam'

Gone with the wind...


In [51]:
ender.alive

False

## Weak References

正是因为有引用，对象才会在内存中存在。当对象的引用计数归零后，垃圾回收程序会把对象销毁。但是，有时需要引用对象，而不让对象存在的时间超过所需时间。

弱引用不会增加对象的引用数量。引用的目标对象称为所指对象（referent）。弱引用不回妨碍所指对象被当作垃圾回收。

In [62]:
import weakref

In [63]:
a_set = {0, 1}
wref = weakref.ref(a_set)

In [64]:
wref

<weakref at 0x10c069050; to 'set' at 0x10bff58c0>

In [65]:
wref()

{0, 1}

In [66]:
a_set = {2, 3, 4}

In [67]:
wref()

{0, 1}

In [68]:
wref() is None

False

In [69]:
wref() is None

False

In [70]:
wref

<weakref at 0x10c069050; to 'set' at 0x10bff58c0>

### The WeakValueDictionary Skit

`WeakValueDictonary`类实现的事一种可变映射，里面的值时对象的弱引用。被引用的对象在程序中的其他地方被当作垃圾回收后，对应的健会自动从`WeakValueDictonary`中删除。因此，`WeakValueDictionary`经常用于缓存。

In [80]:
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
        
    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [81]:
import weakref

In [82]:
stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'), Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese

In [83]:
list(stock.keys())

['Red Leicester', 'Tilsit', 'Brie', 'Parmesan']

In [84]:
del catalog

In [85]:
list(stock.keys())

['Parmesan']

> 临时变量也能用了对象，这可能会导致该变量的存在时间比预期的长。通常，这对局部变量来说不是问题，因为它们在函数返回时会被销毁。但是在示例中，for循环中的变量cheese是全局变量，除非显式删除，否则不会消失。

### Limitations of Weak References

**不是每个Python对象都可以作为弱引用的目标。基本的list和dict实例不能作为所指对象，但是它们的子类可以轻松处解决这个问题。**

In [87]:
class MyList(list):
    """list的子类，实例可以作为弱引用的木笔啊"""

In [88]:
a_list = MyList(range(10))

In [89]:
wref_to_a_list = weakref.ref(a_list)

**set实例可以作为所指对象。用户定义的类型也没问题。int和tuple实例不能作为弱引用的目标，甚至它们的子类也不行。**

## Chaper Summary

每个Python对象都有标识、类型和值。对象的值会随这时间变化。

```
Every Python object has an identify, a type, and a value. Only the value of an object changes over time.
```

变量保存的是引用：

* 简单的赋值不创建副本
* 对`+`后`*`所做的增量赋值来说，如果左边的变量绑定的是不可变对象，会创建新的对象；如果是可变对象，就会改变
* 为现有的变量赋新值，不会修改之前绑定的值。这叫重新绑定：现在变量绑定了其他对象。如果变量是之前那个对象的最后一个引用，对象会被当做垃圾回收
* 函数的参数以别名的形式传递。这意味着，函数可能会修改通过参数传入的可变对象
* **使用可变类型作为函数参数的默认值很危险，因为如果修改了参数，默认值也就变了，这会影响以后使用默认值的调用**

在CPython中，对象的引用计数归零后，对象会被立即销毁。如果除了循环引用之外没有其他引用，两个对象都会被销毁。某些情况下，可能需要保存对象的引用，但不留存对象本身。这个可以通过弱引用实现。